# Met2 Dataset

japanese encoding : https://en.wikipedia.org/wiki/JIS_encoding



In [1]:
import torch
from torch.utils.data import Dataset, DataLoader



In [2]:
import gzip 

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    file_content = f.read()
document = str(file_content, 'EUC-JP')


In [3]:
import gzip

from bs4 import BeautifulSoup

def get_samples(document):
    b_sentence, b_label = [], []

    soup = BeautifulSoup(document, 'html.parser')
    doc_list = list(soup.find_all('doc'))

    for doc in doc_list:
        paragraphs = list(doc.find_all('p'))
        for paragraph in paragraphs:
            p = paragraph

            # labeling
            cursor, sentence, label = 0, [], []

            tag_list = list(p.find_all(['enamex','timex','numex']))
            for tag in tag_list:
                cursor_tag = p.text.find(tag.text, cursor, len(p.text))

                if cursor_tag-cursor>0:
                    sentence.append(p.text[cursor:cursor_tag].replace('\u3000','').replace('\n',''))
                    label.append('None')
                sentence.append(p.text[cursor_tag:cursor_tag+len(tag.text)])
                label.append(tag['type'])

                cursor = cursor_tag+len(tag.text)

            ## paragraph太長了，sentence超過5 當作下一筆sample
            for i in range(0,len(sentence)-5,5):
                b_sentence.append(sentence[i:i+5])
                b_label.append(label[i:i+5])

    return b_sentence, b_label

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    file_content = f.read()
document = str(file_content, 'EUC-JP')

b_sentence, b_label = get_samples(document)

In [4]:
len(b_sentence), len(b_label)

(860, 860)

In [5]:
idx = 859
b_sentence[idx], b_label[idx]

(['カルロス',
  'さんは「',
  'クリスマス',
  'の奇跡だ」と言って絶句したという。現場は機体の破片が長さ１５０メートルにわたって散乱。軍当局が一度は「生存者なし」と確認した後に、次々と生存者が見つかったことで、救助関係者の間に驚きが広がった。',
  'レイエス'],
 ['PERSON', 'None', 'DATE', 'None', 'PERSON'])

## get tags

In [6]:

def get_tags(document):
    tags = set()
    soup = BeautifulSoup(document, 'html.parser')
    name_tags = soup.find_all(['enamex','timex','numex'])
    for tag in name_tags:
        tags.update([tag['type']])
    return ['None']+sorted(list(tags))

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    document = str(f.read(), 'EUC-JP')
    
tags = get_tags(document)
tags

['None',
 'DATE',
 'LOCATION',
 'MONEY',
 'ORGANIZATION',
 'PERCENT',
 'PERSON',
 'TIME']

## get samples

In [7]:
met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    document = str(f.read(), 'EUC-JP')
    
soup = BeautifulSoup(document, 'html.parser')

doc_list = list(soup.find_all('doc'))
#random.shuffle(doc_list)

for doc in doc_list:
    para = doc.find_all('p')
    #print(len(para),para)
    input("")
    for p in para:
        print(p.text)
        data.append(p.text)
        input("")


NameError: name 'random' is not defined

In [ ]:

para = para.find_next('enamex')
para['type'], para.text


In [ ]:
para = para.next_sibling
para

# Cinnamon Dataset

import torch
from torch.utils.data import Dataset, DataLoader

########################################################
##################  Cinnamon Dataset  ##################
class Cinnamon_Dataset(Dataset):
    def __init__(self, cinnamon_path, tokenizer):
        def get_tags(cinnamon_path):
            tags = set()
            files = glob.glob(f'{cinnamon_path}/ca_data/*')
            for file in files:
                dataframe = pd.read_excel(file, encoding="utf8")
                label_str = filter(lambda i:(type(i) is str), dataframe['Tag'])
                def split(strings):
                    out = list()
                    for string in strings: 
                        out += string.split(";")
                    return out
                items = split(label_str)
                tags.update(items)
            return tuple(sorted(list(tags)))
            #return tuple(["[PAD]", "[None]"] + sorted(list(tags)))
        
        def get_samples(cinnamon_path):
            groups = []
            files = glob.glob(f'{cinnamon_path}/ca_data/*')
            for file in files:
                dataframe = pd.read_excel(file, encoding="utf8")

                p_index = dataframe.groupby('Parent Index')
                for g in list(p_index.groups.keys()):
                    groups.append(p_index.get_group(g))
            return groups
        
        self.tokenizer = tokenizer
        self.samples = get_samples(cinnamon_path)
        self.tags = get_tags(cinnamon_path)

        print(f'\t[Info] Load Cannon_Dataset complete !! len:{self.__len__()}')    
        
    def __len__(self):
        return len(self.samples) 
    
    def __getitem__(self, idx):
        return self.samples[idx]
            
    def collate_fn(self, samples):        
        tokenizer, tags = self.tokenizer, self.tags
            
        CLS, SEP, PAD = tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id
        
        def zero_vec(): 
            return [0]*len(tags)
        
        ## text tokenized, label vectoized
        b_token_ids, b_output = [], []
        for sample in samples:
            token_ids = [CLS]
            output = [zero_vec()]
            for text, tag in zip(sample['Text'],sample['Tag']):
                ids = tokenizer.encode(text)[1:-1] + [SEP]
                label = zero_vec()
                if isinstance(tag, str): 
                    for t in tag.split(';'):
                        label[tags.index(t)] = 1
                token_ids += ids
                output += [label]*(len(ids)-1) + [zero_vec()]
            b_token_ids.append(token_ids)
            b_output.append(output)

        ## pad to same lenght
        max_len = min(max([len(s) for s in b_token_ids]), 512)
        for token_ids, output in zip(b_token_ids, b_output):
            token_ids += [PAD]*(max_len-len(token_ids))
            output += [zero_vec()]*(max_len-len(output))

        return torch.tensor(b_token_ids), torch.tensor(b_output)
        
        
from transformers import BertTokenizer 
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)

train_dataset = Cinnamon_Dataset('/media/D/ADL2020-SPRING/project/cinnamon/train/', tokenizer)
valid_dataset = Cinnamon_Dataset('/media/D/ADL2020-SPRING/project/cinnamon/dev/', tokenizer)

train_dataloader = DataLoader(train_dataset,
                             batch_size=8,
                             collate_fn=train_dataset.collate_fn)


In [8]:
import os 
import json
import torch
import torch.nn as nn

from transformers import BertModel

pretrained_weights = 'bert-base-chinese'

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.bert_embedd = BertModel.from_pretrained(pretrained_weights)
        for param in self.bert_embedd.parameters():
            #param.requires_grad = False
            continue 

        #self.dropout = nn.Dropout(0.3)

        hidden_dim = 768
        self.fc = nn.Linear(hidden_dim, 20)

        self.step_loss = dict()
    
    def forward(self, input_ids):
        last_hidden_states, cls_hidden = self.bert_embedd(input_ids)
        output = self.fc(last_hidden_states)
        return output
   
    def save(self, epoch, loss, path='ckpt/'):
        self.step_loss[epoch] = loss 
        with open(f'{path}/step_loss.json', 'w', encoding='utf-8') as f:
            json.dump(self.step_loss, f, indent=4)
        torch.save({'epoch': epoch, 'loss': loss, 'state_dict': self.state_dict()},
                  f'{path}/epoch_{epoch}.pt')
        print(f'\t[Info] save weight, {path}/epoch_{epoch}.pt')

    def load(self, load_file):
        if os.path.isfile(load_file):
            self.load_state_dict(torch.load(load_file)['state_dict'])
            print(f'\t[Info] load weight, {load_file}')
        else:
            print(f'\t[ERROR] {load_file} not exist !')
        return self
        

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.1.1) doesn't match a supported version!
  RequestsDependencyWarning)


In [9]:
import os
import torch
import torch.nn as nn

def _run_train(args, model, criterion, optimizer, dataloader):
    
    model.train()
    
    total_loss, acc, f1 = 0, None, None  
    for idx, (_input, _label) in enumerate(dataloader):
        b = _input.shape[0]
        
        optimizer.zero_grad()
       
        _predict = model(_input.cuda())
        
        loss = criterion(_predict, _label.type_as(_predict).cuda())
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss.item()*b
        #acc, f1 = metrics(_label, _predict)
        print("\t[{}/{}] train loss:{:.3f} ".format(
                                            idx+1,
                                            len(dataloader),
                                            total_loss/(idx+1)/b),
                                    end='   \r')

    return total_loss/len(dataloader.dataset) 
    
def _run_eval(args, model, criterion, dataloader):
 
    model.eval()
    
    with torch.no_grad():
        total_loss, acc, f1 = 0, None, None 
        for idx, (_input, _label) in enumerate(dataloader):
            b = _input.shape[0]

            _predict = model(_input.cuda())

            loss = criterion(_predict, _label.type_as(_predict).cuda())
        
            total_loss += loss.item()*b
            #acc, f1 = metrics(_label, _predict)
            print("\t[{}/{}] valid loss:{:.3f} ".format(
                                            idx+1,
                                            len(dataloader),
                                            total_loss/(idx+1)/b),
                                    end='   \r')     

    return total_loss/len(dataloader.dataset) 

def train(args, train_dataloader, valid_dataloader):
    torch.manual_seed(987)
    torch.cuda.manual_seed(987)
    
    model = Model()
    model.load(args.load_model).cuda() 
    
    criterion = nn.BCEWithLogitsLoss().cuda()#pos_weight=[]).cuda()
     
    optimizer = torch.optim.AdamW(list(model.parameters()), 
                                  lr=args.lr,
                                  eps=1e-8 )

    for epoch in range(1,args.epoch+1):
        print(f' Epoch {epoch}')
            
        loss = _run_train(args, model, criterion, optimizer, train_dataloader)
        print("\t[Info] Train avg loss:{:.4f} ".format(loss))
        
        loss  = _run_eval(args, model, criterion, valid_dataloader)
        print("\t[Info] Valid avg loss:{:.4f} ".format(loss))
        
        ## Save ckpt
        model.save(epoch, loss, args.save_path)
      
        ## Update learning rate
        if epoch>2:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 3
                if param_group['lr'] < 1e-6:
                    param_group['lr'] = 1e-6 

        print('\t--------------------------------------------------------')
    

In [10]:
import torch, json, glob
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class ABC_dataset(Dataset):
    def __init__(self):
        self.data = []

        print(f'\t[Info] Load ABC_Dataset complete !! len:{self.__len__()}')              
    def __len__(self):
        return len(self.data) 
    def __getitem__(self, idx):        
        context  = self.data['context'][idx]
    
def collate_fn(samples):

    return torch.tensor([])

########################################################

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer 
pretrained_weights = 'cl-tohoku/bert-base-japanese-whole-word-masking'

########################################################
##################  Cinnamon Dataset  ##################
class Cinnamon_Dataset(Dataset):
    def __init__(self, cinnamon_path, tokenizer):
        def get_tags(cinnamon_path):
            tags = set()
            files = glob.glob(f'{cinnamon_path}/ca_data/*')
            for file in files:
                dataframe = pd.read_excel(file, encoding="utf8")
                label_str = filter(lambda i:(type(i) is str), dataframe['Tag'])
                def split(strings):
                    out = list()
                    for string in strings: 
                        out += string.split(";")
                    return out
                items = split(label_str)
                tags.update(items)
            return tuple(sorted(list(tags)))
            #return tuple(["[PAD]", "[None]"] + sorted(list(tags)))
        
        def get_samples(cinnamon_path):
            groups = []
            files = glob.glob(f'{cinnamon_path}/ca_data/*')
            for file in files:
                dataframe = pd.read_excel(file, encoding="utf8")

                p_index = dataframe.groupby('Parent Index')
                for g in list(p_index.groups.keys()):
                    groups.append(p_index.get_group(g))
            return groups
        
        self.tokenizer = tokenizer
        self.samples = get_samples(cinnamon_path)
        self.tags = get_tags(cinnamon_path)

        print(f'\t[Info] Load Cannon_Dataset complete !! len:{self.__len__()}')    
        
    def __len__(self):
        return len(self.samples) 
    
    def __getitem__(self, idx):
        return self.samples[idx]
            
    def collate_fn(self, samples):        
        tokenizer, tags = self.tokenizer, self.tags
            
        CLS, SEP, PAD = tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id
        
        def zero_vec(): 
            return [0]*len(tags)
        
        ## text tokenized, label vectoized
        b_token_ids, b_output = [], []
        for sample in samples:
            token_ids = [CLS]
            output = [zero_vec()]
            for text, tag in zip(sample['Text'],sample['Tag']):
                ids = tokenizer.encode(text)[1:-1] + [SEP]
                label = zero_vec()
                if isinstance(tag, str): 
                    for t in tag.split(';'):
                        label[tags.index(t)] = 1
                token_ids += ids
                output += [label]*(len(ids)-1) + [zero_vec()]
            b_token_ids.append(token_ids)
            b_output.append(output)

        ## pad to same lenght
        max_len = min([max([len(s) for s in b_token_ids]), 512])
        for idx,(token_ids, output) in enumerate(zip(b_token_ids, b_output)):            
            token_ids = token_ids[:max_len]
            token_ids += [PAD]*(max_len-len(token_ids))
            b_token_ids[idx] = token_ids
            
            output = output[:max_len]
            output += [zero_vec()]*(max_len-len(output))
            b_output[idx] = output

        return torch.tensor(b_token_ids), torch.tensor(b_output)
    

In [ ]:
#from train import train

import os, warnings, argparse
warnings.filterwarnings('ignore')

#from dataset import *#Cinnamon_Dataset, DataLoader, pretrained_weights


def parse_args(string=None):
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', default=2e-5,
                        type=float, help='leanring rate')
    parser.add_argument('--epoch', default=5,
                        type=int, help='epochs')
    parser.add_argument('--batch-size', default=3,
                        type=int, help='batch size')
    parser.add_argument('--gpu', default="0",
                        type=str, help="0:1080ti 1:1070")
    parser.add_argument('--num-workers', default=2,
                        type=int, help='dataloader num workers')
    parser.add_argument('--cinnamon-data-path', default='/media/D/ADL2020-SPRING/project/cinnamon/',
                        type=str, help='cinnamon dataset')
    parser.add_argument('--load-model', default='ckpt/epoch_6_model_loss_0.4579.pt',
                        type=str, help='.pt model file ')
    parser.add_argument('--save-path', default='ckpt/',
                        type=str, help='.pt model file save dir')
    
    args = parser.parse_args() if string is None else parser.parse_args(string)
    if not os.path.exists(args.save_path): os.makedirs(args.save_path)
    return args

#os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
if __name__=='__main__':
    args = parse_args('')
    
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu 
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    
    ## load tokenizer
    tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)

    ## load dataset
    train_dataset = Cinnamon_Dataset(f'{args.cinnamon_data_path}/train/', tokenizer)
    valid_dataset = Cinnamon_Dataset(f'{args.cinnamon_data_path}/dev/', tokenizer)

    train_dataloader = DataLoader(train_dataset,
                                 batch_size = args.batch_size,
                                 num_workers = args.num_workers,
                                 collate_fn = train_dataset.collate_fn,
                                 shuffle = True)
    valid_dataloader = DataLoader(valid_dataset,
                                 batch_size = args.batch_size*4,
                                 num_workers = args.num_workers,
                                 collate_fn = train_dataset.collate_fn)
    
    ## train
    train(args, train_dataloader, valid_dataloader)
    

	[Info] Load Cannon_Dataset complete !! len:2011
	[Info] Load Cannon_Dataset complete !! len:557
	[ERROR] ckpt/epoch_6_model_loss_0.4579.pt not exist !
 Epoch 1
	[Info] Train avg loss:0.0550  
	[Info] Valid avg loss:0.0181 
	[Info] save weight, ckpt//epoch_1.pt
	--------------------------------------------------------
 Epoch 2
